<img src = 'https://bancariosbh.org.br/manager/resources/uploads/213/350_198_8e66ca00d7dfdfbd00096a7c220be22c.jpg' height="200" width="340">
<p><b><font size = 5 face = 'verdana'> TAREFA AVALIATIVA 5 - PROCESSAMENTO DE LINGUAGEM NATURAL</font></b></p>

<p><font size = 3 face = 'verdana'>Trabalho apresentado à conclusão do curso de PROCESSAMENTO DE LINGUAGEM NATURAL do curso de Pós-Graduação em Inteligência Artificial, PUC Minas, como requisito parcial de avaliação. </font></b></p>
<p><font size = 3 face = 'verdana'><b>PROFESSORA:</b> BARBARA SILVEIRA FRAGA<br> 
<b>ALUNO:</b> OSEMAR DA SILVA XAVIER<br>
<b>MATRÍCULA:</b> 1418586</font></p>
<p><font size = 3 face = 'verdana'>
<hr></hr>

<b>TÓPICOS ABORDADOS:</b><br>
<li>LDA;</li>
<li>SUMARIZAÇÃO;</li>
<li>DESAFIO - WORD EMBEDDINGS + WMD</li>


In [ ]:
# Abaixo faço a importação das bibliotecas que irei utilizar para essa atividade
import yaml
import pandas as pd
import re
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords
import string
import gensim
from gensim import corpora
from os import listdir
from os.path import isfile, join
from bs4 import BeautifulSoup
import requests
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np
import operator
import networkx as nx
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

# LDA


<b> Atividades </b>

1) Faça download dos arquivos no link: https://drive.google.com/open?id=1PgGJ2OBFSKnr3O0RIgO2EvgbLh_avRJN

2) Carregue os quatro documentos. Para carregar um arquivo ``yml``, siga o exemplo abaixo:
```python
with open('data/chats/file.yml') as file:
    chat = yaml.load(file)
```
<b> Dica: adicione os 4 documentos em um único dicionário </b>

In [ ]:
# Fiz a autenticação para o Colab efetuar a leitura 
# dos meus arquivos a partir do Google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Salvei os arquivo no endereçamento abaixo para leitura e apontei para minha variável path
mypath = '/content/drive/MyDrive/PUC Minas/Materias/PROCESSAMENTO DE LINGUAGEM NATURAL/Aula 5/chat/'
files = [f for f in listdir(mypath) if isfile(join(mypath,f))]
# Esses são os arquivos que estão disponíveis no meu diretório
files

['games.yml', 'money.yml', 'proverbs.yml', 'linguistic_knowledge.yml']

In [ ]:
# Verifico o caminho do diretório para atestar que está tudo certo
yaml.safe_load(mypath + files[0])

'/content/drive/MyDrive/PUC Minas/Materias/PROCESSAMENTO DE LINGUAGEM NATURAL/Aula 5/chat/games.yml'

In [ ]:
# Abaixo faço a leitura do primeiro arquivo yml chamado games
with open(mypath + files[0]) as file:
  chat =yaml.safe_load(file)

In [ ]:
# Imprimindo o primeiro arquivo chamado games.yml
chat

{'categories': ['games', 'video games'],
 'conversations': [['Você gosta de algum jogo?',
   'Todos!',
   'tem algum que goste mais?',
   'Não sei te dizer.'],
  ['O que acha do Mario?',
   'Tirando o armário acho bem legal!',
   'Qual seu jogo favorito do Mario?',
   'Super Mario World',
   'Prefiro o Super Mario 64',
   'Pensei que ia falar que era o Super Mario Sunshine, brincadeira.'],
  ['O que acha do Sonic?',
   'Bem legal',
   'Qual seu jogo favorito do Sonic?',
   'Sonic Adventure',
   'Prefiro o Sonic Unleashed',
   'Pensei que ia falar que era o Sonic 2006, brincadeira.'],
  ['O que acha do Pacman?',
   'Pacman? ele é um grande amigo meuQ',
   'Sério?',
   'Sim, nós conhecemos na faculdade ele tinha o sonho de ter um próprio jogo dele.',
   'O que acha dos jogos do Pacman?',
   'Bem divertidos'],
  ["O que acha do Assassin's Creed?",
   'Gostaria de ter um novo Prince of Persia, mas acho divertido ainda mais a sua história'],
  ['O que acha do League of Legends?',
   'Culpa 

In [ ]:
# O próposito é juntar todos os arquivos yml
# Abaixo faço um for para unir os arquivos
all_chats = {}
for f in files:
  with open(mypath + f) as file:
    chat = yaml.safe_load(file)
    all_chats[chat['categories'][0]] = chat['conversations']

In [ ]:
# Todos os arquivos foram agrupados
all_chats

{'games': [['Você gosta de algum jogo?',
   'Todos!',
   'tem algum que goste mais?',
   'Não sei te dizer.'],
  ['O que acha do Mario?',
   'Tirando o armário acho bem legal!',
   'Qual seu jogo favorito do Mario?',
   'Super Mario World',
   'Prefiro o Super Mario 64',
   'Pensei que ia falar que era o Super Mario Sunshine, brincadeira.'],
  ['O que acha do Sonic?',
   'Bem legal',
   'Qual seu jogo favorito do Sonic?',
   'Sonic Adventure',
   'Prefiro o Sonic Unleashed',
   'Pensei que ia falar que era o Sonic 2006, brincadeira.'],
  ['O que acha do Pacman?',
   'Pacman? ele é um grande amigo meuQ',
   'Sério?',
   'Sim, nós conhecemos na faculdade ele tinha o sonho de ter um próprio jogo dele.',
   'O que acha dos jogos do Pacman?',
   'Bem divertidos'],
  ["O que acha do Assassin's Creed?",
   'Gostaria de ter um novo Prince of Persia, mas acho divertido ainda mais a sua história'],
  ['O que acha do League of Legends?',
   'Culpa do Jungler',
   'Ele é feeder?',
   'Não só quero

In [ ]:
# Visualizando todas as chaves criadas no dicionário
all_chats.keys()

dict_keys(['games', 'money', 'proverbs', 'conhecimento linguístico'])

In [ ]:
# Posso viasualizar cada uma, caso queira
# Abaixo peço para imprimir o conteúdo do keys 'money'
all_chats['money']

[['Você é pago', 'Não tenho necessidade de dinheiro.'],
 ['mercado de ações', 'comprar baixo, vender alto.'],
 ['mercado de ações', 'investir em si mesmo.'],
 ['mercado de ações', 'Por que não simplesmente levar tudo para um cassino?'],
 ['mercado de ações', 'Eu não recomendaria comprar na margem.'],
 ['mercado de ações', 'você nunca pode realmente prever o mercado de ações.'],
 ['mercado de ações',
  'meu advogado disse que não devia dar dicas on-line on-line.'],
 ['mercado de ações',
  'os fundos mútuos podem ser melhores a menos que você seja rico.'],
 ['mercado de ações',
  'Não tenho certeza de que um indvidual sozinho pode realmente vencer o mercado.'],
 ['taxa de juros', 'tudo depende das ações do banco central.'],
 ['o que é um dólar', 'dólar é uma unidade de moeda nos Estados Unidos.'],
 ['o que é dinheiro',
  'peças padrão de ouro, prata, cobre, níquel, etc. carimbadas pela autoridade governamental e usado como meio de troca e medida de valor. qualquer substância ou artigo us

<b> Atividade </b>

A variável ``chat`` é um dicionário, que possui a chave ``conversations``, que contém várias listas. Dentro de cada lista, existe um par de frases, uma escrita por um humano e outra escrita por um robô. Neste exercício, só iremos considerar a frase do humano, ou seja, a primeira posição do par nas listas.
Exemplo:

```python
chat = {'categories': ['compliment'],
        'conversations': [['você é linda', 'eu sei'],
                          ['adoro você', 'também'],
                          ['gosto da sua sinceridade', 'que bom']]}
```
No exemplo acima, iremos considerar apenas as frases: <b> 'você é linda', 'adoro você' e 'gosto da sua sinceridade' </b>.

Um documento no LDA é representado por uma única lista. Ou seja, as listas de frases selecionadas no exemplo anterior se tornarão uma única lista: <b> ['você é linda adoro você e gosto da sua sinceridade'] </b>



3) Conforme explicado anteriormente, faça as alterações necessárias no dado. Primeiro, considere apenas a frase da primeira posição, depois una todas as listas de frases em uma única lista.

In [ ]:
# Transformando todos o texto em string
for c in all_chats:
	all_chats[c] = ''.join([chat[0] for chat in all_chats[c]])

In [ ]:
all_chats

{'games': "Você gosta de algum jogo?O que acha do Mario?O que acha do Sonic?O que acha do Pacman?O que acha do Assassin's Creed?O que acha do League of Legends?O que acha do Dota 2?Devo comprar um console?Mario ou Sonic?Conhece um programa chamado Discord?Conhece um programa chamado Team Speak?Conhece um programa chamado Skype?Você joga em consoles?Você acompanha o cenário de e-Sports?Você prefere PC ou console?",
 'money': 'Você é pagomercado de açõesmercado de açõesmercado de açõesmercado de açõesmercado de açõesmercado de açõesmercado de açõesmercado de açõestaxa de juroso que é um dólaro que é dinheiroo que é o mercado de açõeso que é o mercado de açõeso que é o mercado de açõesQual é o seu investimento favorito?o que é economiao que é economiao que é economiaeu faço estoquedinheiroquanto é que você ganhaquanto é que você ganhaquanto é que você ganhaquanto você cobraquanto dinheiro você temquanto dinheiroquanto dinheiro1 dólar1 realquem é o dono de um público',
 'proverbs': 'Na cas

<b> Atividade </b>

4) Faça uma limpeza nos dados carregados e processados do passo anterior. Aplique:
- tokenização
- lower
- remoção stopwords
- remoção pontuação

Ao final você deve obter uma lista com todos os documentos e em cada documento as palavras tokenizadas, exemplo:

```python
[['oi', 'tudo', 'bem'],
 ['mario', 'game', 'jogo']]
```

In [ ]:
# Nessa etapa fiz o pré processamento dos dados
def pre_processamento_texto(corpus):
  corpus_alt = re.findall(r'\w+(?:\w+)?|[^\w\s]', corpus)
  corpus_alt = [t.lower() for t in corpus_alt]
  portugues_stops = stopwords.words('portuguese')
  corpus_alt = [t for t in corpus_alt if t not in portugues_stops]
  corpus_alt = [re.sub(r'\d','',t) for t in corpus_alt]
  corpus_alt =[t for t in corpus_alt if t not in string.punctuation]
  corpus_alt = [unicode(t) for t in corpus_alt]
  return corpus_alt

In [ ]:
# Fazendo o download das stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Foi necessário fazer a importação do sys porque estava dando erro 
# 'unicode' is not defined
import sys
if sys.version_info[0] >= 3:
    unicode = str

In [ ]:
# Fiz o processamento dos dados dentro da variável lista chats_limpos
chats_limpos = []
for c in all_chats:
  chats_limpos.append(pre_processamento_texto(all_chats[c]))

In [ ]:
chats_limpos

[['gosta',
  'algum',
  'jogo',
  'acha',
  'mario',
  'acha',
  'sonic',
  'acha',
  'pacman',
  'acha',
  'assassin',
  's',
  'creed',
  'acha',
  'league',
  'of',
  'legends',
  'acha',
  'dota',
  'devo',
  'comprar',
  'console',
  'mario',
  'sonic',
  'conhece',
  'programa',
  'chamado',
  'discord',
  'conhece',
  'programa',
  'chamado',
  'team',
  'speak',
  'conhece',
  'programa',
  'chamado',
  'skype',
  'joga',
  'consoles',
  'acompanha',
  'cenário',
  'sports',
  'prefere',
  'pc',
  'console'],
 ['pagomercado',
  'açõesmercado',
  'açõesmercado',
  'açõesmercado',
  'açõesmercado',
  'açõesmercado',
  'açõesmercado',
  'açõesmercado',
  'açõestaxa',
  'juroso',
  'dólaro',
  'dinheiroo',
  'mercado',
  'açõeso',
  'mercado',
  'açõeso',
  'mercado',
  'açõesqual',
  'investimento',
  'favorito',
  'economiao',
  'economiao',
  'economiaeu',
  'faço',
  'estoquedinheiroquanto',
  'ganhaquanto',
  'ganhaquanto',
  'ganhaquanto',
  'cobraquanto',
  'dinheiro',
  'te

<b> Atividade </b>

5) Crie um dicionário com nosso corpus, de forma que cada termo único (valor) possua um índice (chave). 

Dica: 
- utilize o ```gensim.corpora.Dictionary ``` para criar este dicionário. 
- utilize o texto pré-processado

In [ ]:
# Usando o gensim.corpora.Dictionary para criar este dicionário]
# uma vez que o LDA visualiza as palavras como bag of words
dicionario = gensim.corpora.Dictionary(chats_limpos)

In [ ]:
# Verificando o tamanho do dicionario criado
# Resumindo, criamos 245 tokens
len(dicionario)

247

In [ ]:
# Verificando a palavra na posição 4 por exemplo
# Dentro dicionário o ID 4 é respectivo a palavra 'cenário'
dicionario[4]

'cenário'

<b> Atividade </b>

6) Converta os documentos em índices, utilizando o método ```doc2bow``` em cada documento.

In [ ]:
# Abaixo foi feito a conversão dos indices utilizando o método doc2bow
chat_doc2bow = [dicionario.doc2bow(chat) for chat in chats_limpos]

In [ ]:
# Imprimindo o resultado
chat_doc2bow

[[(0, 6),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 3),
  (6, 1),
  (7, 3),
  (8, 2),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 2),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 3),
  (25, 1),
  (26, 1),
  (27, 2),
  (28, 1),
  (29, 1),
  (30, 1)],
 [(31, 7),
  (32, 2),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 2),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 2),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 3),
  (48, 1),
  (49, 1),
  (50, 3),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1)],
 [(6, 2),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 2),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 1),
  (79, 1),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 1),
  (84, 1),
  (85, 1),
  (86, 1),
  (87, 1),
  (88, 2),
  (89, 1),
  (90, 1

<b> Atividade </b>

7) Aplique o modelo LDA do Gensim nos dados processados acima. O modelo recebe a matriz de índices gerada anteriormente.
Abaixo, apresentamos um exemplo de como aplicar o modelo.


```python
model = gensim.models.ldamodel.LdaModel(doc_term_matrix, num_topics=4, id2word = dictionary, passes=1000, random_state=123)
#para mostrar os tópicos:
model.show_topics()
```

Fique a vontade para alterar os parâmetros e fazer testes.

In [ ]:
# Aplicando o modelo LDA do Gensim nos dados processados
# No num_topics é possível alterar esse valor para trazer a quantidade 
# de topicos que necessito para minha aplicação
model = gensim.models.ldamodel.LdaModel(chat_doc2bow, num_topics=4, id2word = dicionario, passes=1000, random_state=123)


In [ ]:
# Gerei 4 tópicos para visualização
model.show_topics()

[(0,
  '0.047*"açõesmercado" + 0.021*"quais" + 0.021*"mercado" + 0.021*"ganhaquanto" + 0.015*"uso" + 0.015*"porquê" + 0.015*"linguística" + 0.015*"dinheiro" + 0.015*"economiao" + 0.015*"açõeso"'),
 (1,
  '0.059*"acha" + 0.030*"chamado" + 0.030*"programa" + 0.030*"conhece" + 0.021*"console" + 0.021*"mario" + 0.021*"sonic" + 0.012*"team" + 0.012*"pc" + 0.012*"of"'),
 (2,
  '0.004*"economiaeu" + 0.004*"favorito" + 0.004*"cobraquanto" + 0.004*"dinheiroo" + 0.004*"dinheiroquanto" + 0.004*"dono" + 0.004*"dólar" + 0.004*"dólaro" + 0.004*"açõesqual" + 0.004*"estoquedinheiroquanto"'),
 (3,
  '0.015*"ter" + 0.011*"comprar" + 0.011*"·" + 0.011*"vou" + 0.011*"bem" + 0.011*"conseguiu" + 0.006*"ferroganhei" + 0.006*"faça" + 0.006*"feito" + 0.006*"feitome"')]

<b> Atividade </b>

8) Para responder esta questão utilize os parâmetros sugeridos na questão 7. Analise os grupos gerados anteriormente e responda:
- a) Os grupos são bem dividos? Caracterize-os.<br>
<b>R. Considerando o 4 tópicos sim. O primeiro caracteriza por abordar mercado financeiro, o segundo games, o terceiro economia e o quarto faz menções de compras.</b>
- b) Dê um nome para cada grupo de tópicos.<br>
<b>R.:Financeiro, Games, Economia e Compras</b>

# Sumarização

Aqui iremos realizar uma sumarização de notíticas com técnicas que vimos nas aulas anteriores.

<b> Atividade </b>

9) Faça download do texto da seguinte notícia: 

``https://g1.globo.com/tecnologia/noticia/2022/03/12/instagram-restrito-na-russia-entenda-a-importancia-da-rede-social-para-o-pais-de-putin.ghtml``

Dica: para fazer download, consulte a Aula Prática 4.

In [ ]:
# Fiz o download da notícia abaixo
url = 'https://g1.globo.com/tecnologia/noticia/2022/03/12/instagram-restrito-na-russia-entenda-a-importancia-da-rede-social-para-o-pais-de-putin.ghtml'

In [ ]:
# Faz a captura do texto do link contido na variável url
def match_class(target):                                                        
    def do_match(tag):                                                          
        classes = tag.get('class', [])                                          
        return all(c in classes for c in target)                                
    return do_match 

def get_text_url(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    #remove marcações de scripts e style
    texto = soup.find_all(match_class(["content-text__container"]))
    all_text = ""
    for t in texto:
        all_text += t.get_text()
    return all_text

In [ ]:
noticia = get_text_url(url)

<b> Atividade </b>

10) Imprima o texto coletado.

In [ ]:
# Fazendo a impressão do texto coletado
noticia

' A decisão de bloquear o Instagram na Rússia deve ter impactos que vão além da disputa entre o governo russo e a companhia americana, já que o aplicativo é bastante popular no país. A rede social tem 80 milhões de usuários russos, segundo o diretor Adam Mosseri.  A Rússia anunciou que vai restringir o acesso ao Instagram na próxima segunda-feira (14) como resposta à mudança de política para discurso de ódio da Meta, controladora do Instagram e também das plataformas Facebook e WhatsApp.  Na última sexta-feira (11), a empresa americana mudou temporariamente suas regras para permitir que usuários das redes sociais em alguns países defendam atos de violência contra russos no contexto da guerra na Ucrânia (saiba mais ao fim da reportagem).  De acordo com o comunicado do governo russo, a decisão de bloquear o app na segunda dá aos usuários 48 horas para transferirem suas fotos e vídeos para outras redes sociais e notificarem seus contatos e clientes.  Os russos usam muito mais o Instagram 

<b> Atividade </b>

11) Faça a tokenização das sentenças.

In [ ]:
# Gerando os token, mas para isso é necessário baixar o nltk.download('punkt')
nltk.download('punkt')
noticia_sents = nltk.sent_tokenize(noticia)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Fazendo o len, temos como resultado 24 frases
len(noticia_sents)

24

<b> Atividade </b>

12) Aplique o pré-processamento nos dados:

- tokenização
- lower
- remoção stopwords
- remoção pontuação

In [ ]:
# Para cada uma frase das 24 será necessário aplicar pré processamento dos dados
def pre_processamento_texto(corpus):
  corpus_alt = re.findall(r'\w+(?:\w+)?|[^\w\s]', corpus)
  corpus_alt = [t.lower() for t in corpus_alt]
  portugues_stops = stopwords.words('portuguese')
  corpus_alt = [t for t in corpus_alt if t not in portugues_stops]
  corpus_alt = [re.sub(r'\d','',t) for t in corpus_alt]
  corpus_alt =[t for t in corpus_alt if t not in string.punctuation]
  corpus_alt = [unicode(t) for t in corpus_alt]
  return corpus_alt

In [ ]:
# Fiz um for para a aplicação do pre processamento e 
# coloquei dentro da variavel noticia_limpa
noticia_limpa = [pre_processamento_texto(sent) for sent in noticia_sents]

In [ ]:
# Imprimindo os tokens gerados
noticia_limpa

[['decisão',
  'bloquear',
  'instagram',
  'rússia',
  'deve',
  'ter',
  'impactos',
  'vão',
  'além',
  'disputa',
  'governo',
  'russo',
  'companhia',
  'americana',
  'aplicativo',
  'bastante',
  'popular',
  'país'],
 ['rede',
  'social',
  'milhões',
  'usuários',
  'russos',
  'segundo',
  'diretor',
  'adam',
  'mosseri'],
 ['rússia',
  'anunciou',
  'vai',
  'restringir',
  'acesso',
  'instagram',
  'próxima',
  'segunda',
  'feira',
  'resposta',
  'mudança',
  'política',
  'discurso',
  'ódio',
  'meta',
  'controladora',
  'instagram',
  'plataformas',
  'facebook',
  'whatsapp'],
 ['última',
  'sexta',
  'feira',
  'empresa',
  'americana',
  'mudou',
  'temporariamente',
  'regras',
  'permitir',
  'usuários',
  'redes',
  'sociais',
  'alguns',
  'países',
  'defendam',
  'atos',
  'violência',
  'contra',
  'russos',
  'contexto',
  'guerra',
  'ucrânia',
  'saiba',
  'fim',
  'reportagem'],
 ['acordo',
  'comunicado',
  'governo',
  'russo',
  'decisão',
  'bloq

<b> Atividade </b>

13) Agora iremos implementar a sumarização. Para sumarizar você precisará implementar um método para cada uma das orientações abaixo:

13.1) Faça um método que calcule a similaridade entre duas sentenças utilizando a similaridade do conseno. 

Dica: faça a tranformação aplicando Bag of Words. Consulte a Aula 3.

In [ ]:
bag = CountVectorizer()

In [ ]:
f0 = noticia_limpa[0]
f1 = noticia_limpa[1]

In [ ]:
# Para gerar os bag of words é necessário que f0 e f1
# estejam convertidas em string
' '.join(f0),' '.join(f1)

('decisão bloquear instagram rússia deve ter impactos vão além disputa governo russo companhia americana aplicativo bastante popular país',
 'rede social milhões usuários russos segundo diretor adam mosseri')

In [ ]:
# Gerando os bag of words das palavras 
# Gerou uma matrix de 2 linhas e 27 colunas
bag.fit_transform([' '.join(f0),' '.join(f1)])

<2x27 sparse matrix of type '<class 'numpy.int64'>'
	with 27 stored elements in Compressed Sparse Row format>

In [ ]:
# Apliquei tudo na minha variável vetor
vetor = bag.fit_transform([' '.join(f0),' '.join(f1)])

In [ ]:
# Para visualizar o vetor da posição zero
v0 = vetor.todense()[0]

In [ ]:
# Para visualizar o vetor da posição um
v1 = vetor.todense()[1]

In [ ]:
# De posse desses 2 vetores é possível calcular a similaridade de coseno
# O resultado array([[0.]]) pode ser interpretado que não existem
# Similaridade entre v0 e v1
cosine_similarity(v0,v1)

array([[0.]])

In [ ]:
# Se compararmos a v0 com v0 podemos notar que o valor será array([[1.]])
# Porque são idênticas possuem a mesma frase
cosine_similarity(v0,v0)

array([[1.]])

In [ ]:
# Como temos 24 frases, teríamos que calcular uma para cada
# Aplicamos abaixo o método
def calcula_similaridade(f0,f1):
  bag = CountVectorizer()
  vetor = bag.fit_transform([' '.join(f0), ' '.join(f1)])
  return cosine_similarity(vetor[0],vetor[1])[0][0]

In [ ]:
calcula_similaridade(noticia_limpa[20],noticia_limpa[20])

0.9999999999999999

13.2) Faça um método para construir uma matriz de similaridades entre as sentenças. 

Exemplo, seu texto tem 10 sentenças, você deve construir uma matriz (10,10). A posição (0,1) da matriz corresponderá a similaridade do cosseno entre a sentença 0 e a sentença 1.

In [ ]:
# Temos um total de 24 frases na variável noticia_limpa
# Para montarmos nossa matrix o ideal é que ela possua 24 X 24
len(noticia_limpa)

24

In [ ]:
# A seguir criaremos uma matriz de zeros
np.zeros((len(noticia_limpa), len(noticia_limpa))).shape

(24, 24)

In [ ]:
# Criando uma matriz de zeros, para em seguida criar um for para preenchimento da matriz
matriz_similaridade = np.zeros((len(noticia_limpa), len(noticia_limpa)))
matriz_similaridade

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 

In [ ]:
# Para isso vamos utilizar um for
for i in range(0, len(noticia_limpa)):
	for j in range(0, len(noticia_limpa)):
		if i == j:
			continue
		matriz_similaridade[i][j] = calcula_similaridade(noticia_limpa[i], noticia_limpa[j])

In [ ]:
# A seguir transformamos nossa matriz de similaridade em um dataframe
df_matriz_similaridade = pd.DataFrame(matriz_similaridade)

13.3) - Depois que construímos a matriz de similaridades, faça o ranking das sentenças utilizando o método ``pagerank`` do pacote `networkx`. Este método irá retornar um score para cada sentença. Para utilizar este método, os dados devem representados em um grafo. Para isto converta a matriz de similaridades para um grafo de similaridades, utilizando o método `network.from_numpy_array(matrix)`.

<i>Mais informações Pagerank: </i>https://www.agenciamestre.com/marketing-digital/o-que-e-pagerank/

In [ ]:
# Existem 2 forma para 
# Criei a média de similaridade de cada uma das frases
# Considerando as 5 maiores similaridades 
df_matriz_similaridade.mean().sort_values(ascending=False).head(5)

7     0.126687
2     0.123209
8     0.103992
5     0.103742
17    0.102269
dtype: float64

In [ ]:
# Criei a mediana da similaridade de cada uma das frases
# Considerando as 5 maiores similaridades 
df_matriz_similaridade.median().sort_values(ascending=False).head(5)

7     0.105409
2     0.100973
0     0.097659
16    0.074032
18    0.070286
dtype: float64

13.4) Ordene os scores retornados pelo `pagerank` e imprima as <b> 5 </b> sentenças com maior score.

In [ ]:
# Fazendo a impressão dos maiores scores
index_media = sorted(df_matriz_similaridade.mean().sort_values().index[-5:])
index_media

[2, 5, 7, 8, 17]

In [ ]:
# Criei a sumarização das 5 frases mais relevantes
' '.join([noticia_sents[i] for i in index_media])

'A Rússia anunciou que vai restringir o acesso ao Instagram na próxima segunda-feira (14) como resposta à mudança de política para discurso de ódio da Meta, controladora do Instagram e também das plataformas Facebook e WhatsApp. Os russos usam muito mais o Instagram do que o Facebook, a rede social com mais usuários no mundo, segundo dados da consultoria eMarketer publicados pela agência France Presse. O Instagram não divulga dados de usuários por país, mas, na sexta, Mosseri postou que são 80 milhões na Rússia. "Essa decisão cortará 80 milhões na Rússia uns dos outros e do resto do mundo, já que cerca de 80% das pessoas na Rússia seguem uma conta do Instagram fora de seu país. A decisão é uma resposta à mudança de política sobre discurso de ódio da Meta, controladora do Facebook, do WhatsApp e do Instagram.'

In [ ]:
# ranking das sentenças utilizando o método pagerank do pacote networkx
grafo_matrix = nx.from_numpy_array(matriz_similaridade)
grafo_matrix

In [ ]:
# Imprime score
scores_page_rank = nx.pagerank(grafo_matrix)

In [ ]:
scores_sorted = dict(sorted(scores_page_rank.items(), key=operator.itemgetter(1),reverse=True))

In [ ]:
index_page_rank = list(scores_sorted.keys())[:5]
index_page_rank

[7, 2, 17, 5, 8]

In [ ]:
# Criei a sumarização das 5 frases mais relevantes
' '.join([noticia_sents[i] for i in index_page_rank])

'O Instagram não divulga dados de usuários por país, mas, na sexta, Mosseri postou que são 80 milhões na Rússia. A Rússia anunciou que vai restringir o acesso ao Instagram na próxima segunda-feira (14) como resposta à mudança de política para discurso de ódio da Meta, controladora do Instagram e também das plataformas Facebook e WhatsApp. A decisão é uma resposta à mudança de política sobre discurso de ódio da Meta, controladora do Facebook, do WhatsApp e do Instagram. Os russos usam muito mais o Instagram do que o Facebook, a rede social com mais usuários no mundo, segundo dados da consultoria eMarketer publicados pela agência France Presse. "Essa decisão cortará 80 milhões na Rússia uns dos outros e do resto do mundo, já que cerca de 80% das pessoas na Rússia seguem uma conta do Instagram fora de seu país.'

<b> Atividade </b>

14) Analise o resultado da sumarização, construindo uma crítica sobre a qualidade e relevância da mesma.
<br><b>R.: Considerando os métodos utilizandomédia ou o Pagerank, os resultados são iguais (2, 5, 7, 8 e 17). Dessa forma os 2 métodos são bem eficientes. </b>

<b> Atividade </b>

15) Faça o exercício 13 mais uma vez, agora substituindo a representação BoW pela TFIDF, usando o TDIDFVectorizer

<b> Atividade </b>

16) Faça uma comparação dos resultados de sumarização obtidos pelas técnicas de Bag of Words e TFIDF, explicitando as diferenças observadas de forma concreta e exemplificando suas análises com frases dos textos.

<b> Desafio! (opcional, valendo pontos extras) </b>

Implemente e analise os resultados de uma solução que faça a sumarização dos textos utilizando Word Embeddings + WMD